In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from ansim_dataset_unconf import ansimDataset, create_circular_mask
from ConvLSTM_unconf import MtConvLSTM
import random
import math
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import time
import os

In [2]:
img_path = '/home/rliu/ansim/data/unconfined_cropped/'
img_list_csv = '/home/rliu/github/ansim/unconfined/img_list.csv'
train_csv = '/home/rliu/github/ansim/unconfined/train_unconf.csv'
test_csv = '/home/rliu/github/ansim/unconfined/test_unconf.csv'
output_path = '/home/rliu/ansim/models/dataset3/very_first.pt'

In [3]:
trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=10, random_rotate = True, transform=None, rand_range=10)
trainloader = torch.utils.data.DataLoader(trainset,
                                             batch_size=8, shuffle=True,
                                             num_workers=4)

testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=10, random_rotate = False, transform=None)
testloader = torch.utils.data.DataLoader(testset,
                                             batch_size=8, shuffle=False,
                                             num_workers=4)

In [4]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("GPU in use")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GPU in use


In [8]:
img_list = pd.read_csv(img_list_csv,header=None)

In [43]:
os.path.join(img_path, img_list.iloc[114570, 0])

'/home/rliu/ansim/data/data/JPEGImages/114571.jpg'

In [53]:
image = Image.open("/home/rliu/ansim/data/data/JPEGImages/112893.jpg")
image = image.convert('L')

In [6]:
# def train_model(encoder, decoder, criterion, optimizer, scheduler, num_epochs=25):
#     since = time.time()

#     best_encoder_wts = encoder.state_dict()
#     best_decoder_wts = decoder.state_dict()
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training phase
#         scheduler.step()
#         encoder.train(True)  # Set model to training mode
#         decoder.train(True)  # Set model to training mode
#         running_loss = 0.0
#         running_corrects = 0

#         # Iterate over data.
#         trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
#         trainloader = torch.utils.data.DataLoader(trainset,
#                                                      batch_size=1, shuffle=True,
#                                                      num_workers=4)

#         print("trainloader ready!")
#         testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
#         testloader = torch.utils.data.DataLoader(testset,
#                                                      batch_size=1, shuffle=True,
#                                                      num_workers=4)
#         print("testloader ready!")
        
#         for data in trainloader:
#             # get the inputs
#             data_split = torch.split(data, 10, dim=1)
#             inputs = data_split[0]
#             target = data_split[1]
# #            print(inputs)
#             # wrap them in Variable
#             if use_gpu:
# #                inputs = Variable(inputs.cuda())
# #                labels = Variable(labels.cuda())
# #                inputs = torch.nn.DataParallel(inputs, device_ids=[0, 1]).cuda()
# #                labels = torch.nn.DataParallel(labels, device_ids=[0, 1]).cuda()
#                 inputs, target = inputs.to(device), target.to(device)
# #                print(inputs)
#             else:
#                 inputs, target = Variable(inputs), Variable(target)

#             # zero the parameter gradients
#             optimizer.zero_grad()
            
#             output, h, c, states = encoder(inputs)
#             output_last = output[0][0].double()
#             h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#             c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#             states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#             x = decoder.activateConv(states_cat)
#             input_d = [x, states]
#             output_d, h_d, c_d, states_d = decoder(input_d)
#             predicted = torch.cat(output_d, dim=0, out=None).double()
            
            
#             loss = criterion(predicted, target)
#             # forward
# #            outputs = model(inputs)
# #            _, preds = torch.max(outputs.data, 1)
# #            loss = criterion(outputs, labels)

#             loss.backward()
#             optimizer.step()

#             # statistics
#             iter_loss = loss.item()
#             running_loss += loss.item()    
#             epoch_loss = running_loss / len(trainset)
            
#             print('{} Loss: {:.4f} batch_loss: {:d}'.format(
#                 "train", epoch_loss, iter_loss))
        
#         with torch.no_grad():
#             for data in testloader:
#                 data_split = torch.split(data, 10, dim=1)
#                 inputs = data_split[0]
#                 target = data_split[1]
                
#                 if use_gpu:
#                     inputs, target = inputs.to(device), target.to(device)
#                 else:
#                     inputs, target = Variable(inputs), Variable(target)

                    
#                 output, h, c, states = encoder(inputs)
#                 output_last = output[0][0].double()
#                 h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#                 c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#                 states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#                 x = decoder.activateConv(states_cat)
#                 input_d = [x, states]
#                 output_d, h_d, c_d, states_d = decoder(input_d)
#                 predicted = torch.cat(output_d, dim=0, out=None).double()
            
#                 loss_test = criterion(predicted, target)
#                 iter_loss_test = loss_test.item()
#                 running_loss_test += loss_test.item()    
#                 epoch_loss_test = running_loss_test / len(testset)

#         print('Loss on the test images: %.5f %%' % (
#             epoch_loss_test))
        

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     encoder.load_state_dict(best_encoder_wts)
#     decoder.load_state_dict(best_decoder_wts)
#     return encoder, decoder

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, batch_size = 4, step_size = 10):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training phase
        scheduler.step()
        model.train(True)  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0
        

        # Iterate over data.
        trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=10, random_rotate = True, transform=None, image_size = 128, rand_range=10)
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True,
                                                     num_workers=1)

        print("trainloader ready!")
        testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=10, random_rotate = False, transform=None, image_size = 128, rand_range=0)
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False,
                                                     num_workers=1)
        print("testloader ready!")
        
        for data in trainloader:
            # get the inputs
            data_split = torch.split(data, int(data.shape[1]/2), dim=1)
            inputs = data_split[0]
            target = data_split[1]
#            print(inputs)
            # wrap them in Variable
            if use_gpu:
                inputs, target = inputs.to(device), target.to(device)
            else:
                inputs, target = Variable(inputs), Variable(target)

            # zero the parameter gradients
            optimizer.zero_grad()
            
#             output, h, c, states = encoder(inputs)
#             output_last = output[0][0].double()
#             h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#             c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#             states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#             x = decoder.activateConv(states_cat)
#             input_d = [x, states]
#             output_d, h_d, c_d, states_d = decoder(input_d)
#             predicted = torch.cat(output_d, dim=0, out=None).double()
            
            _, _, _, predicted = model(inputs)
            
#             m = nn.Sigmoid()
#             loss = criterion(m(predicted), m(target))
            # forward
#            outputs = model(inputs)
#            _, preds = torch.max(outputs.data, 1)
            loss = criterion(predicted, target)

            loss.backward()
            optimizer.step()

            # statistics
            iter_loss = loss.item()
            running_loss += loss.item()    
            epoch_loss = running_loss / len(trainset)
            
            print('{} Loss: {:.4f} batch_loss: {:f}'.format(
                "train", epoch_loss, iter_loss))
        
        with torch.no_grad():
            running_loss_test = 0.0
            for data in testloader:
                data_split = torch.split(data, int(data.shape[1]/2), dim=1)
                inputs = data_split[0]
                target = data_split[1]
                
                
                if use_gpu:
                    inputs, target = inputs.to(device), target.to(device)
                else:
                    inputs, target = Variable(inputs), Variable(target)

                    
#                 output, h, c, states = encoder(inputs)
#                 output_last = output[0][0].double()
#                 h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#                 c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#                 states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#                 x = decoder.activateConv(states_cat)
#                 input_d = [x, states]
#                 output_d, h_d, c_d, states_d = decoder(input_d)
#                 predicted = torch.cat(output_d, dim=0, out=None).double()

                _, _, _, predicted = model(inputs)
                
#                 m = nn.Sigmoid()
#                 loss_test = criterion(m(predicted), m(target))
                loss_test = criterion(predicted, target)
    
                iter_loss_test = loss_test.item()
                running_loss_test += loss_test.item()    
                epoch_loss_test = running_loss_test / len(testset)

        print('Loss on the test images: %.5f ' % (
            epoch_loss_test))
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return encoder, decoder

In [ ]:
# transfer learning resnet18
step_size = 10
model = MtConvLSTM(input_size=(128,128),
                 input_dim=1,
                 hidden_dim=[[16,32,64],[16,32,64],[32,64,128],[32,64,128,128]],
                 kernel_size=[[3,3,3],[5,3,3],[5,5,5],[7,5,5,5]],
                 num_layers=[3,3,3,4],
                 predict_steps=5,
                 batch_first=True,
                 num_scale=4,
                 bias=True,
                 return_all_layers=True)


if use_gpu:
#     encoder = torch.nn.DataParallel(encoder)
#     decoder = torch.nn.DataParallel(decoder)
    model = torch.nn.DataParallel(model)
    model.to(device)

criterion = nn.MSELoss()
# criterion = nn.BCELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

# train model
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, batch_size = 1, step_size = 10, num_epochs=10)
torch.save(model, output_path)

Epoch 0/9
----------
trainloader ready!
testloader ready!
train Loss: 129.5743 batch_loss: 23323.371094
train Loss: 268.7171 batch_loss: 25045.708984
train Loss: 431.5089 batch_loss: 29302.521484
train Loss: 568.6107 batch_loss: 24678.328125
train Loss: 638.9578 batch_loss: 12662.476562
train Loss: 732.9921 batch_loss: 16926.167969
train Loss: 828.9702 batch_loss: 17276.064453
train Loss: 921.4092 batch_loss: 16639.019531
train Loss: 1067.4052 batch_loss: 26279.285156
train Loss: 1159.3100 batch_loss: 16542.865234
train Loss: 1226.7843 batch_loss: 12145.374023
train Loss: 1323.6583 batch_loss: 17437.304688
train Loss: 1388.8048 batch_loss: 11726.372070
train Loss: 1479.5061 batch_loss: 16326.240234
train Loss: 1583.4307 batch_loss: 18706.427734
train Loss: 1754.5836 batch_loss: 30807.519531
train Loss: 1868.4319 batch_loss: 20492.693359
train Loss: 1963.6807 batch_loss: 17144.779297
train Loss: 2024.5729 batch_loss: 10960.605469
train Loss: 2101.2278 batch_loss: 13797.880859
train Loss

train Loss: 10203.5891 batch_loss: 2785.458740
train Loss: 10215.1835 batch_loss: 2086.995361
train Loss: 10228.5885 batch_loss: 2412.901123
train Loss: 10242.7579 batch_loss: 2550.486816
Loss on the test images: 3424.14242 
Epoch 1/9
----------
trainloader ready!
testloader ready!
train Loss: 23.0630 batch_loss: 4151.342285
train Loss: 33.8237 batch_loss: 1936.918701
train Loss: 39.2681 batch_loss: 979.990417
train Loss: 51.1958 batch_loss: 2146.992920
train Loss: 81.9032 batch_loss: 5527.324707
train Loss: 100.5045 batch_loss: 3348.239746
train Loss: 115.5514 batch_loss: 2708.449951
train Loss: 147.5361 batch_loss: 5757.231445
train Loss: 152.7874 batch_loss: 945.240906
train Loss: 157.5952 batch_loss: 865.401855
train Loss: 167.8228 batch_loss: 1840.971436
train Loss: 200.7730 batch_loss: 5931.031738
train Loss: 214.4202 batch_loss: 2456.498779
train Loss: 219.6804 batch_loss: 946.843750
train Loss: 230.5404 batch_loss: 1954.798096
train Loss: 261.6949 batch_loss: 5607.811523
train 

In [ ]:
# testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=step_size, random_rotate = True, mask = mask, transform=None)
# testloader = torch.utils.data.DataLoader(testset,
#                                              batch_size=1, shuffle=True,
#                                              num_workers=1)
# dataiter = iter(testloader)
# data = dataiter.next()
# data_split = torch.split(data, int(data.shape[1]/2), dim=1)
# inputs = data_split[0]
# target = data_split[1]
# outputs = model(inputs)


In [ ]:
count_parameters(model)

In [10]:
m = nn.Sigmoid()
input = Variable(torch.randn(3, 1, 512, 64, 32)).cuda()
a = m(input)

In [11]:
a.shape

torch.Size([3, 1, 512, 64, 32])